## Part 1

In [75]:
struct Maze
    m::Array{Char, 2}
end

function Maze(s::Vector{T}, padding = 1, default = '#') where T <: String 
    m = hcat(collect.(s)...)
    h, w = size(m)
    m0 = fill(default, h + 2*padding, w + 2*padding)
    m0[(padding + 1):(h + padding), (padding + 1):(w + padding)] .= m
    Maze(m0)
end

find_element(m::Maze, el::Char) = CartesianIndices(m.m)[m.m .== el]

function show_maze(m::Maze)
    for i in 1:size(m.m)[1]
        for j in 1:size(m.m)[2]
            print(m.m[i, j])
        end
        print('\n')
    end
end


function neighbours(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[m.m[l + d] == '.' for d in dirs]]]
end

function teleports(m::Maze)
    entry_points = Dict{Symbol, Vector{CartesianIndex}}()
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    for t in 'A':'Z'
        for loc in find_element(m, t)
            entry_point = CartesianIndex(-1, -1)
            for d in dirs
                if m.m[loc + d] in collect('A':'Z')
                    global teleport_name = Symbol(join(sort([t, m.m[loc + d]])))
                elseif m.m[loc + d] == '.'
                    entry_point = loc + d
                end
            end
            if entry_point != CartesianIndex(-1, -1)
                points = get!(entry_points, teleport_name, [])
                push!(points, entry_point)
            end
        end
    end
    
    tlps = Dict{CartesianIndex, CartesianIndex}()
    for (k, v) in entry_points
        if k in [:AA, :ZZ] continue end
        tlps[v[1]] = v[2]
        tlps[v[2]] = v[1]
    end
    
    entry_points, tlps
end

function floodfill(m::Maze, tlps, start)
    arr = fill(-1, size(m.m)...)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
                if (n in keys(tlps))
                    if (arr[tlps[n]] == -1) || (arr[tlps[n]] > arr[loc] + 2)
                        arr[tlps[n]] = arr[loc] + 2
                        push!(acc, tlps[n])
                    end
                end
            end
        end
    end
    
    arr
end

floodfill (generic function with 2 methods)

In [87]:
function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    tlps = teleports(m)
    ffm = floodfill(m, tlps[2], tlps[1][:AA][1])
    ffm[tlps[1][:ZZ][1]]
end

part1 (generic function with 2 methods)

In [90]:
println("Part 1: ", part1("input.txt"))

Part 1: 654


In [84]:
m = Maze(readlines("test2.txt"))

Maze(['#' '#' … '#' '#'; '#' ' ' … ' ' '#'; … ; '#' ' ' … ' ' '#'; '#' '#' … '#' '#'])

In [67]:
show_maze(m)

#####################
#        B    D F   #
#        C    E G   #
#  ######.####.#.#  #
#  ######.####...#  #
#  ######...######  #
#  ########.######  #
#  ########.######  #
#  #####   D #####  #
#  #####   E #####  #
#AA.....BC   #####  #
#  #.###     #####  #
#  #.###   FG..###  #
#  #.###     #.###  #
#  #.###     #....ZZ#
#  #.###########.#  #
#  #.###########.#  #
#  #.###########.#  #
#  #.............#  #
#  ###############  #
#                   #
#                   #
#####################


In [85]:
res = teleports(m)

(Dict(:AS => [CartesianIndex(19, 10), CartesianIndex(34, 19)],:JP => [CartesianIndex(17, 36), CartesianIndex(23, 30)],:CP => [CartesianIndex(23, 10), CartesianIndex(21, 36)],:AA => [CartesianIndex(21, 4)],:NY => [CartesianIndex(28, 15), CartesianIndex(4, 25)],:DI => [CartesianIndex(10, 23), CartesianIndex(4, 17)],:JO => [CartesianIndex(4, 21), CartesianIndex(15, 30)],:FL => [CartesianIndex(17, 30), CartesianIndex(34, 23)],:ZZ => [CartesianIndex(4, 19)],:BU => [CartesianIndex(13, 36), CartesianIndex(28, 23)]…), Dict{CartesianIndex,CartesianIndex}(CartesianIndex(23, 30) => CartesianIndex(17, 36),CartesianIndex(28, 19) => CartesianIndex(34, 25),CartesianIndex(21, 36) => CartesianIndex(23, 10),CartesianIndex(23, 10) => CartesianIndex(21, 36),CartesianIndex(15, 30) => CartesianIndex(4, 21),CartesianIndex(4, 17) => CartesianIndex(10, 23),CartesianIndex(34, 23) => CartesianIndex(17, 30),CartesianIndex(4, 25) => CartesianIndex(28, 15),CartesianIndex(34, 19) => CartesianIndex(19, 10),CartesianI

In [81]:
res[1][:ZZ][1]

CartesianIndex(15, 18)

In [86]:
ffm = floodfill(m, res[2], res[1][:AA][1])
ffm[res[1][:ZZ][1]]

58

In [83]:
ffm[res[1][:ZZ][1]]

23

In [51]:
ffm[res[2]['C'][1]]

-1

In [23]:
res[3]

Dict{CartesianIndex,CartesianIndex} with 6 entries:
  CartesianIndex(8, 12)  => CartesianIndex(4, 15)
  CartesianIndex(4, 10)  => CartesianIndex(11, 8)
  CartesianIndex(13, 14) => CartesianIndex(4, 17)
  CartesianIndex(4, 15)  => CartesianIndex(8, 12)
  CartesianIndex(4, 17)  => CartesianIndex(13, 14)
  CartesianIndex(11, 8)  => CartesianIndex(4, 10)

## Part 2

In [145]:
function teleports(m::Maze)
    entry_points = Dict{Symbol, Vector{CartesianIndex}}()
    nest_level = Dict{CartesianIndex, Int}()
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    h, w = size(m.m)
    for t in 'A':'Z'
        for loc in find_element(m, t)
            entry_point = CartesianIndex(-1, -1)
            for d in dirs
                if m.m[loc + d] in collect('A':'Z')
                    global teleport_name = Symbol(join(sort([t, m.m[loc + d]])))
                elseif m.m[loc + d] == '.'
                    entry_point = loc + d
                end
            end
            if entry_point != CartesianIndex(-1, -1)
                points = get!(entry_points, teleport_name, [])
                push!(points, entry_point)
                if (loc[1] == 3) || (loc[1] == h - 2) || (loc[2] == 3) || (loc[2] == w - 2)
                    nest_level[entry_point] = -1
                else
                    nest_level[entry_point] = 1
                end
            end
        end
    end
    
    tlps = Dict{CartesianIndex, CartesianIndex}()
    for (k, v) in entry_points
        if k in [:AA, :ZZ] continue end
        tlps[v[1]] = v[2]
        tlps[v[2]] = v[1]
    end
    
    entry_points, nest_level, tlps
end

function floodfill(m::Maze, tlps, nest_levels, start, finish)
    arr = Dict{Tuple{CartesianIndex, Int}, Int}()
    arr[start] = 0
    acc = [start]
    VERYBIG = 2^63 - 1
    arr[finish] = VERYBIG
    while !isempty(acc)
        loc, level = popfirst!(acc)
        for n in neighbours(m, loc)
            val = get!(arr, (n, level), VERYBIG)
            if (val > arr[(loc, level)] + 1) && (arr[(loc, level)] + 1 < arr[finish])
                arr[(n, level)] = arr[(loc, level)] + 1
                push!(acc, (n, level))
                if (n in keys(tlps))
                    if (get!(arr, (tlps[n], level + nest_levels[n]), VERYBIG) > arr[(loc, level)] + 2) &&
                            (arr[(loc, level)] + 2 < arr[finish]) &&
                            (level + nest_levels[n] >= 0)
                        arr[(tlps[n], level + nest_levels[n])] = arr[(loc, level)] + 2
                        push!(acc, (tlps[n], level + nest_levels[n]))
                    end
                end
            end
        end
    end
    
    arr
end

floodfill (generic function with 4 methods)

In [146]:
function part2(inp = "input.txt")
    m = Maze(readlines(inp))
    # show_maze(m)
    res = teleports(m)
    ffm = floodfill(m, res[3], res[2], (res[1][:AA][1], 0), (res[1][:ZZ][1], 0))
    println("Part 2: ", ffm[(res[1][:ZZ][1], 0)])
end

part2 (generic function with 2 methods)

In [132]:
m = Maze(readlines("test.txt"))
show_maze(m)
res = teleports(m)
ffm = floodfill(m, res[3], res[2], (res[1][:AA][1], 0))

#####################
#        B    D F   #
#        C    E G   #
#  ######.####.#.#  #
#  ######.####...#  #
#  ######...######  #
#  ########.######  #
#  ########.######  #
#  #####   D #####  #
#  #####   E #####  #
#AA.....BC   #####  #
#  #.###     #####  #
#  #.###   FG..###  #
#  #.###     #.###  #
#  #.###     #....ZZ#
#  #.###########.#  #
#  #.###########.#  #
#  #.###########.#  #
#  #.............#  #
#  ###############  #
#                   #
#                   #
#####################
CartesianIndex(11, 4):0
CartesianIndex(11, 5):0
CartesianIndex(11, 6):0
CartesianIndex(12, 5):0
CartesianIndex(11, 7):0
CartesianIndex(13, 5):0
CartesianIndex(11, 8):0
CartesianIndex(4, 10):1
CartesianIndex(14, 5):0
CartesianIndex(5, 10):1
CartesianIndex(15, 5):0
CartesianIndex(6, 10):1
CartesianIndex(16, 5):0
CartesianIndex(6, 11):1
CartesianIndex(17, 5):0
CartesianIndex(6, 12):1
CartesianIndex(18, 5):0
CartesianIndex(7, 12):1
CartesianIndex(19, 5):0
CartesianIndex(8, 12):1
CartesianIndex

Dict{Tuple{CartesianIndex,Int64},Int64} with 78 entries:
  (CartesianIndex(16, 5), 0)   => 6
  (CartesianIndex(15, 16), 0)  => 26
  (CartesianIndex(7, 12), 1)   => 10
  (CartesianIndex(6, 10), 1)   => 7
  (CartesianIndex(5, 15), 2)   => 13
  (CartesianIndex(11, 6), 0)   => 2
  (CartesianIndex(4, 10), 1)   => 5
  (CartesianIndex(8, 12), 1)   => 11
  (CartesianIndex(5, 17), 0)   => 9223372036854775807
  (CartesianIndex(17, 17), 0)  => 23
  (CartesianIndex(19, 15), 0)  => 19
  (CartesianIndex(11, 7), 0)   => 3
  (CartesianIndex(13, 14), -1) => 9223372036854775807
  (CartesianIndex(13, 5), 0)   => 3
  (CartesianIndex(15, 18), 0)  => 26
  (CartesianIndex(19, 17), 0)  => 21
  (CartesianIndex(13, 15), 1)  => 18
  (CartesianIndex(8, 12), 0)   => 36
  (CartesianIndex(16, 17), 0)  => 24
  (CartesianIndex(11, 8), 0)   => 4
  (CartesianIndex(13, 15), 0)  => 29
  (CartesianIndex(19, 5), 0)   => 9
  (CartesianIndex(17, 5), 0)   => 7
  (CartesianIndex(5, 17), 2)   => 15
  (CartesianIndex(13, 14), 0) 

In [136]:
ffm[(res[1][:ZZ][1], 0)]

26

In [149]:
part2("input.txt")

Part 2: 7360


In [101]:
res[3]

Dict{CartesianIndex,CartesianIndex} with 6 entries:
  CartesianIndex(8, 12)  => CartesianIndex(4, 15)
  CartesianIndex(4, 10)  => CartesianIndex(11, 8)
  CartesianIndex(13, 14) => CartesianIndex(4, 17)
  CartesianIndex(4, 15)  => CartesianIndex(8, 12)
  CartesianIndex(4, 17)  => CartesianIndex(13, 14)
  CartesianIndex(11, 8)  => CartesianIndex(4, 10)

In [113]:
VERYBIG = 12345678901234

12345678901234

In [114]:
typeof(VERYBIG)

Int64

In [119]:
typeof(2^63)

Int64

In [121]:
2^63 - 1

9223372036854775807